In [1]:
from scraper import number_companies, valuetoday_usa_scraper, df_cleaning, save_csv

In [2]:
number_companies()

5083

In [8]:
dataframe = valuetoday_usa_scraper(5083)
final_df = df_cleaning(dataframe)

IndexError: list index out of range

In [6]:
final_df

,Company Name,Company Business,Market Value,Annual Revenue,Operating Income,Net Income,Assets,Liabilities
0,APPLE,Technology,2256000000000,274510000000,66280000000,57410000000,317344000000,245062000000
1,MICROSOFT CORPORATION,Technology,1682000000000,143000000000,52950000000,44300000000,301311000000,183007000000
2,ALPHABET,Technology,1185000000000,182520000000,41220000000,40260000000,299243000000,86320000000
3,AMAZON.COM,eCommerce,1634000000000,386060000000,22890000000,21330000000,110908000000,184586000000
4,FACEBOOK,Communication Services,778040000000,85960000000,32670000000,29150000000,139691000000,29244000000
...,...,...,...,...,...,...,...,...
135,ILLUMINA,Healthcare,54020000000,3543000000,0,1002000000,7248000000,2685000000
136,EOG RESOURCES,Energy,29093000000,17122000000,0,2734000000,35222000000,14834000000
137,METLIFE,Financial Services,42253000000,69620000000,0,5899000000,768000000,691890000000
138,EDWARDS LIFESCIENCES CORPORATION,Healthcare,56859000000,4348000000,0,1047000000,6424000000,2585000000


In [4]:
save_csv(final_df, "value_today")

In [1]:
import requests
import re
import pandas as pd
import warnings
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
from time import sleep

In [3]:
ua = UserAgent()
header = {'User-Agent':str(ua.chrome)}

collected_names = []
collected_businesses = []
collected_values = []
collected_values_metrics = []
collected_revenues = []
collected_revenues_metrics = []
collected_op_incomes = []
collected_op_incomes_metrics = []
collected_net_incomes = []
collected_net_incomes_metrics = []
collected_assets = []
collected_assets_metrics = []
collected_liabilities = []
collected_liabilities_metrics = []

url = "https://www.value.today/headquarters/united-states-america-usa?page="

for page in range(66, 67):

    #print(page)

    source = requests.get(url + str(page), headers=header)
    soup = BeautifulSoup(source.content, "html.parser")

    # Create a list of divs with all required information.
    info_blocks = soup.find_all("div", class_="node node--type-listed-companies node--view-mode-teaser ds-2col-stacked clearfix")

    for item in info_blocks:

        def collect_values(item, collected_list):
            """Takes in parsed div tag, name of particular list and append extracted or None values to that list."""
            if len(item) == 0 or len(item) >= 2:
                collected_list.append(None)
            else:
                string = item[0].text.strip() 
                value = re.findall("\d+[.,]\d+|\\b(?<![a-zA-Z]-)\d+\\b|(?<!\S)-\d+.\d+", string)[0]
                collected_list.append(value)


        def collect_metrics(item, collected_list):
            """Takes in parsed div tag, name of particular list and append extracted or None values to that list."""
            if len(item) == 0 or len(item) >= 2:
                collected_list.append(None)
            else:
                string = item[0].text.strip()
                if "Billion" in string:
                    collected_list.append("Billion")
                if "Million" in string:
                    collected_list.append("Million")

        
        market_value_item = item.select("div.field--name-field-market-value-jan012021 > div.field--item")
        if len(market_value_item) == 0:
            continue
        else:
            market_value = market_value_item[0]["content"]
            collected_values.append(market_value)
            collect_metrics(market_value_item, collected_values_metrics)

        name = item.find("h2").text.strip()
        collected_names.append(name)
        print(name)

        business = item.select("div.field--name-field-company-category-primary > div > div.field--item > a")[0].text.strip()
        collected_businesses.append(business)
        #print(len(collected_businesses))

        revenue_item = item.select("div.field--name-field-annual-revenue > div.field--item")
        collect_values(revenue_item, collected_revenues)
        collect_metrics(revenue_item, collected_revenues_metrics)
        print(collected_revenues_metrics)

        op_income_item = item.select("div.field--name-field-annual-operating-income > div.field--item")
        collect_values(op_income_item, collected_op_incomes)
        collect_metrics(op_income_item, collected_op_incomes_metrics)
        #print(len(collected_op_incomes_metrics))

        net_income_item = item.select("div.field--name-field-annual-net-income-lc > div.field--item")
        collect_values(net_income_item, collected_net_incomes)
        collect_metrics(net_income_item, collected_net_incomes_metrics)
        #print(len(collected_net_incomes_metrics))

        assets_item = item.select("div.field--name-field-total-assets > div.field--item")
        collect_values(assets_item, collected_assets)
        collect_metrics(assets_item, collected_assets_metrics)
        #print(len(collected_assets_metrics))

        liabilities_item = item.select("div.field--name-field-total-liabilities- > div.field--item")
        collect_values(liabilities_item, collected_liabilities)
        collect_metrics(liabilities_item, collected_liabilities_metrics)
        #print(len(collected_liabilities_metrics))

    sleep(2)
    #print(collected_revenues_metrics)
#df = pd.DataFrame({"Company Name": collected_names, "Company Business": collected_businesses, "Market Value": collected_values, 
                   # "Value Metric": collected_values_metrics, "Annual Revenue": collected_revenues, 
                   # "Revenue Metric": collected_revenues_metrics, "Operating Income": collected_op_incomes, 
                   # "Op Income Metric": collected_op_incomes_metrics, "Net Income": collected_net_incomes, 
                   # "Net Income Metric": collected_net_incomes_metrics, "Assets": collected_assets, 
                   # "Assets Metric": collected_assets_metrics, "Liabilities": collected_liabilities, 
                   # "Liabilities Metric": collected_liabilities_metrics})

A. O. SMITH CORPORATION
[None]
PENTAIR
[None, None]
POST HOLDINGS
[None, None, 'Million']
ABIOMED
[None, None, 'Million', 'Million']
PVH
[None, None, 'Million', 'Million']
SPIRIT AEROSYSTEMS HOLDINGS
[None, None, 'Million', 'Million', None]
ERIE INDEMNITY COMPANY
[None, None, 'Million', 'Million', None, 'Million']
COMMERCE BANCSHARES
[None, None, 'Million', 'Million', None, 'Million', None]
DROPBOX
[None, None, 'Million', 'Million', None, 'Million', None, None]
HILL-ROM HOLDINGS
[None, None, 'Million', 'Million', None, 'Million', None, None, 'Million']


In [37]:
df

,Company Name,Company Business,Market Value,Value Metric,Annual Revenue,Revenue Metric,Operating Income,Op Income Metric,Net Income,Net Income Metric,Assets,Assets Metric,Liabilities,Liabilities Metric
0,RELIANCE STEEL & ALUMINUM,Basic Materials,7.635,Billion,None,None,None,None,None,None,None,None,None,None
1,ARES CAPITAL CORPORATION,Financial Services,7.138,Billion,None,None,None,None,None,None,None,None,None,None
2,KOHL'S CORPORATION,Consumer Cyclical,6.42,Billion,"15,955",Million,-262,Million,-163,Million,None,None,None,None
3,MONOLITHIC POWER SYSTEMS,Technology,16.516,Billion,844.4,Million,166.6,Million,164.3,Million,None,None,None,None
4,BRUKER CORPORATION,Healthcare,8.288,Billion,2020,Billion,None,None,None,None,None,None,None,None
5,ALBEMARLE CORPORATION,Basic Materials,15.705,Billion,3,Million,505.8,Million,375.7,Million,None,None,None,None
6,APARTMENT INVESTMENT AND MANAGEMENT COMPANY,Real Estate,0.786,Billion,None,None,None,None,None,None,None,None,None,None
7,IPG PHOTONICS CORPORATION,Technology,11.928,Billion,1,Million,231.5,Million,159.5,Million,None,None,None,None
8,DOUGLAS EMMETT,Real Estate,5.117,Billion,None,None,None,None,None,None,None,None,None,None


In [27]:
df[df["Company Name"] == "YOUTUBE"]

,Company Name,Company Business,Market Value,Value Metric,Annual Revenue,Revenue Metric,Operating Income,Op Income Metric,Net Income,Net Income Metric,Assets,Assets Metric,Liabilities,Liabilities Metric
3,YOUTUBE,Social Media,None,None,None,None,None,None,None,None,None,None,None,None


In [80]:
info_blocks[2]

<div class="node node--type-listed-companies node--view-mode-teaser ds-2col-stacked clearfix" data-history-node-id="46201">
<div class="clearfix group-header">
<div class="field field--name-node-title field--type-ds field--label-hidden field--item"><h2 class="text-primary">
<a href="/company/post-holdings" hreflang="en">POST HOLDINGS</a>
</h2>
</div>
<div class="clearfix col-sm-6 field field--name-field-world-rank-jan012021 field--type-integer field--label-above">
<div class="field--label">World Rank (Jan-01-2021)</div>
<div class="field--item">2346</div>
</div>
<div class="clearfix col-sm-6 field field--name-field-market-value-jan012021 field--type-float field--label-above">
<div class="field--label">Market Value (Jan-01-2021)</div>
<div class="field--item" content="6.655">6.655 Billion USD</div>
</div>
<div class="clearfix col-sm-6 field field--name-field-world-rank-jan-2020 field--type-integer field--label-above">
<div class="field--label">World Rank (Jan-2020)</div>
<div class="fie

In [6]:
data = pd.read_csv("value_today.csv")

In [7]:
data

,Company Name,Company Business,Market Value,Annual Revenue,Operating Income,Net Income,Assets,Liabilities
0,APPLE,Technology,2256000000000,274510000000,66280000000,274510000000,317344000000,245062000000
1,MICROSOFT CORPORATION,Technology,1682000000000,143000000000,52950000000,143000000000,301311000000,183007000000
2,ALPHABET,Technology,1185000000000,182520000000,41220000000,182520000000,299243000000,86320000000
3,AMAZON.COM,eCommerce,1634000000000,386060000000,22890000000,386060000000,110908000000,184586000000
4,FACEBOOK,Communication Services,778040000000,85960000000,32670000000,85960000000,139691000000,29244000000
5,BERKSHIRE HATHAWAY,Financial Services,543679999999,327220000000,0,327220000000,829946000000,410730000000
6,J P MORGAN CHASE & CO,Financial Services,387330000000,122900000000,0,122900000000,3386000000000,3106000000000
7,VISA,Financial Services,482170000000,21840000000,14090000000,21840000000,77884000000,42360000000
8,JOHNSON & JOHNSON,Healthcare,414310000000,82580000000,19730000000,82580000000,158380000000,95402000000
9,WALMART,Retail,407840000000,523960000000,22540000000,523960000000,237382000000,162072000000


In [54]:
string = "5,698 Million USD  for Year ending September-2020"

if "Billion" in string:
    print("Billion")
if "Million" in string:
    print("Million")

True
